In [2]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0


In [7]:
import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import random
import pandas as pd
import re
import unicodedata

# Load the trained model
model = load_model("my_model.h5")

# Reinitialize Tokenizer
tokenizer = Tokenizer(oov_token="<OOV>")

# Load and preprocess dataset to fit the tokenizer
df = pd.read_csv("Roman-Urdu-Poetry (1).csv")

def clean_text(text):
    if pd.isna(text):
        return ""
    text = ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))
    text = re.sub(r"[^a-zA-Z0-9\s.,?!]", "", text)
    text = re.sub(r"\.(?=\w)", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.lower()

df["Poetry"] = df["Poetry"].apply(clean_text)
tokenizer.fit_on_texts(df['Poetry'])

# Define max sequence length
max_sequence_length = 20

# Poem Generation Function
def generate_poem(prompt, num_lines, words_per_line, temperature):
    poem = []
    current_word = prompt.lower()

    for _ in range(num_lines):
        line = current_word

        for _ in range(words_per_line - 1):
            token_list = tokenizer.texts_to_sequences([line])[0]
            token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')

            predictions = model.predict(token_list, verbose=0)[0]
            predictions = np.log(predictions + 1e-10) / temperature
            exp_preds = np.exp(predictions)
            predictions = exp_preds / np.sum(exp_preds)

            sorted_indices = np.argsort(predictions)[-5:]
            possible_words = [tokenizer.index_word.get(idx, None) for idx in sorted_indices if idx in tokenizer.index_word]
            possible_words = [word for word in possible_words if word is not None]

            if possible_words:
                word = random.choices(possible_words, weights=predictions[sorted_indices])[0]
            else:
                break

            line += " " + word
            current_word = word

        poem.append(line.capitalize())

    return "\n".join(poem)

# Custom CSS
custom_css = """
body {
    background-color: #121212;
    font-family: 'Poppins', sans-serif;
}
.gradio-container {
    max-width: 700px !important;
    margin: auto;
    padding: 20px;
    border-radius: 12px;
    background: #1e1e1e;
    box-shadow: 0px 0px 15px rgba(255, 255, 255, 0.1);
}
h1, h2 {
    color: #ffffff;
    text-align: center;
}
.gradio-interface .input-text, .gradio-interface .input-number, .gradio-interface .input-slider {
    background: #292929;
    color: #ffffff;
    border: 1px solid #555;
    border-radius: 10px;
    padding: 10px;
}
button {
    background: linear-gradient(135deg, #ff8a00, #e52e71);
    color: white;
    font-size: 16px;
    padding: 12px;
    border-radius: 8px;
    border: none;
    cursor: pointer;
    transition: 0.3s;
}
button:hover {
    transform: scale(1.05);
    box-shadow: 0px 0px 10px rgba(255, 138, 0, 0.5);
}
"""

# Gradio Interface
demo = gr.Interface(
    fn=generate_poem,
    inputs=[
        gr.Textbox(label="Enter Starting Word", placeholder="Type a word..."),
        gr.Number(label="Number of Lines", value=5, precision=0),
        gr.Number(label="Words per Line", value=5, precision=0),
        gr.Slider(label="Temperature (Creativity)", minimum=0.1, maximum=2.0, value=1.0, step=0.1)
    ],
    outputs=gr.Textbox(label="Generated Poem"),
    title="🌙 AI Roman Urdu Poetry Generator 🎭",
    description="Generate AI-powered Roman Urdu poetry! Just enter a starting word, set poem parameters, and let the AI craft poetic magic.",
    theme="default",
    css=custom_css
)

# Launch Gradio App
demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://edfc3d8ffbcece3e33.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
